In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

Pedro had an idea for how to get matches out of the initial song, so he pulled my iniT_model, as well as some api_requests, and then used it to create a matched_df. This was then implemented into our app.

In [ ]:
import pandas as pd
from test_package.modules.PCA import pca
data = pd.read_csv("raw_data/tracks_features.csv")

In [ ]:
from test_package.modules.api_request import api_request
from test_package.modules.nn_model import fit_nn
from test_package.modules.PCA import pca
from test_package.modules.preprocessing import preprocess_data,preprocess_numerical


def init_model(data,
               data_upcoming,
               n_components=7):
    """
    Initialize the model and return the model object
    """
    preproc_num = preprocess_numerical(data)
    preproc_up_num = preprocess_numerical(data_upcoming)
    data_pca, data_pca_up = pca(preproc_num, preproc_up_num, n_components)
    song_knn = fit_nn(data_pca_up, len(preproc_up_num))

    return song_knn, data_pca_up

In [ ]:
song_knn, data_pca = init_model(data,data.iloc[:100])
song_knn

In [ ]:
s1 = pd.Series(song_knn.kneighbors(data_pca[0].reshape(1,-1),return_distance=True)[0][0])
s2 = pd.Series(song_knn.kneighbors(data_pca[0].reshape(1,-1),return_distance=True)[1][0])

In [ ]:
pd.concat([s1,s2],axis=1)

In [ ]:
def find_artist(data,
                song_name,
                artist_name,
                n_neighbors):
    """
    Finds {n_neighbors} artists that sound like a given song
    """
    model, data_proc = init_model(data, n_neighbors)

    song_index = data.index[(data.name == song_name ) & (data.artists.str.strip('[]').str.strip("'") == artist_name)][0]

    similar_artist = model.kneighbors(data_proc[song_index].reshape(1, -1),
                                         return_distance=False)

    return similar_artist

In [ ]:
def find_gig(data,            
             song_name,
             artist_name,
             n_neighbors):
    """
    Find the proposed gigs based on a given song
    """

    gig_list = []

    similar_artist = find_artist(data,
                                 song_name,
                                 artist_name,
                                 n_neighbors)

    for artist in similar_artist:
        gig = api_request(artist)

        if gig == 0:
            gig_list.append('No gigs found for this artist')
        else:
            gig_list.append(gig)

    return gig_list

In [ ]:
from test_package.modules.PCA import pca
data_num = data.select_dtypes(exclude = ['object']).drop(columns = ['track_number', 'disc_number'])
prep= preprocess_data(data_num)
pca1, pca2 = pca(prep, prep,7)

In [ ]:
preprocess_numerical(data).shape